# Predict farm comodity prices using deep learned models.

## Pre-proccess data

In [2]:
%matplotlib inline  

import matplotlib
import pandas as pd
from numpy import nan
import numpy as np
from datetime import datetime
from dateutil.parser import parse
import arrow
import numpy
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction import DictVectorizer as DV
from keras.preprocessing.text import Tokenizer
import csv
from keras.models import Sequential
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


hello
hello


In [9]:
raw_data  = pd.read_csv("data.csv").drop("Commodity_Name",axis = 1)
print(raw_data.head())
raw_data = raw_data[raw_data["Price"]>0]
raw_data[['date','month','year']] = raw_data['Date'].str.split("-",expand=True)
raw_data["date"]= raw_data.apply(lambda row: int(row['date']),axis = 1)
raw_data["month"]= raw_data.apply(lambda row: int(row['month']),axis = 1)
raw_data["year"]= raw_data.apply(lambda row: int(row['year']),axis = 1)
raw_data = raw_data.drop("Date",axis =1)
raw_data["Centre_Name"] = raw_data["Centre_Name"].astype('category')
raw_data["Centre_Name_cat"] = raw_data["Centre_Name"].cat.codes
raw_data.to_csv("processed.csv",quoting=csv.QUOTE_NONNUMERIC)


       Date Centre_Name  Price
0  01-01-09  CHANDIGARH    NaN
1  01-01-09       DELHI    NaN
2  01-01-09      SHIMLA    NaN
3  01-01-09    SRINAGAR    NaN
4  01-01-09       JAMMU    NaN


In [4]:
DataScalerx = MinMaxScaler()
DataScalery = MinMaxScaler()
msk = np.random.rand(len(raw_data)) < 0.8
raw_data_train = raw_data[msk]
raw_data_test = raw_data[~msk]
x = DataScalerx.fit_transform(X=raw_data.drop(["Centre_Name","Price"],axis=1),y=raw_data["Price"])
y = DataScalery.fit_transform(X=raw_data["Price"].values.reshape(-1, 1)) 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(38146, 4)
(38146, 1)
(9537, 4)
(9537, 1)


In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
import keras.backend as K
def mean_pred(y_true, y_pred):
    return K.mean(y_pred - y_true)
from keras import losses
import keras
import h5py

In [6]:
model = Sequential()
#input layer
model.add(Dense(8, input_shape=x_train[0].shape))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.4))

# hidden layers
model.add(Dense(8))
model.add(BatchNormalization())
model.add(Activation("sigmoid"))
model.add(Dropout(0.4))
    
model.add(Dense(4))
model.add(BatchNormalization())
model.add(Activation("sigmoid"))
model.add(Dropout(0.4))
    
model.add(Dense(2, activation="sigmoid"))
    
# output layer
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop',  metrics=['mae'])

In [8]:
model.fit(x_train, y_train, epochs=15, batch_size=20,validation_split=0.05)
model.save("hello.h5")

Train on 36238 samples, validate on 1908 samples
Epoch 1/15
36238/36238 [==============================] - 14s - loss: 0.0111 - mean_absolute_error: 0.0808 - val_loss: 0.0234 - val_mean_absolute_error: 0.1113
Epoch 2/15
36238/36238 [==============================] - 12s - loss: 0.0108 - mean_absolute_error: 0.0798 - val_loss: 0.0230 - val_mean_absolute_error: 0.1140
Epoch 3/15
36238/36238 [==============================] - 15s - loss: 0.0106 - mean_absolute_error: 0.0791 - val_loss: 0.0228 - val_mean_absolute_error: 0.1183
Epoch 4/15
36238/36238 [==============================] - 14s - loss: 0.0106 - mean_absolute_error: 0.0788 - val_loss: 0.0232 - val_mean_absolute_error: 0.1124
Epoch 5/15
36238/36238 [==============================] - 15s - loss: 0.0104 - mean_absolute_error: 0.0782 - val_loss: 0.0232 - val_mean_absolute_error: 0.1228
Epoch 6/15
36238/36238 [==============================] - 14s - loss: 0.0105 - mean_absolute_error: 0.0783 - val_loss: 0.0228 - val_mean_absolute_error

In [10]:
model=keras.models.load_model("hello.h5")
y_test_predict = model.predict(x_test)
DataScalery.inverse_transform(y_test_predict)

array([[ 2446.07397461],
       [ 2448.20776367],
       [ 2448.72705078],
       ..., 
       [ 1441.97668457],
       [ 1707.81994629],
       [ 1488.2154541 ]], dtype=float32)

In [11]:
model2 = model
model2.fit(x_train, y_train, epochs=25, batch_size=20,validation_split=0.05)
model2.save("hello.h5")

Train on 36238 samples, validate on 1908 samples
Epoch 1/25
36238/36238 [==============================] - 14s - loss: 0.0101 - mean_absolute_error: 0.0768 - val_loss: 0.0238 - val_mean_absolute_error: 0.1282
Epoch 2/25
36238/36238 [==============================] - 13s - loss: 0.0101 - mean_absolute_error: 0.0768 - val_loss: 0.0243 - val_mean_absolute_error: 0.1301
Epoch 3/25
36238/36238 [==============================] - 15s - loss: 0.0100 - mean_absolute_error: 0.0765 - val_loss: 0.0238 - val_mean_absolute_error: 0.1272
Epoch 4/25
36238/36238 [==============================] - 13s - loss: 0.0101 - mean_absolute_error: 0.0770 - val_loss: 0.0236 - val_mean_absolute_error: 0.1255
Epoch 5/25
36238/36238 [==============================] - 13s - loss: 0.0100 - mean_absolute_error: 0.0763 - val_loss: 0.0233 - val_mean_absolute_error: 0.1276
Epoch 6/25
36238/36238 [==============================] - 13s - loss: 0.0100 - mean_absolute_error: 0.0764 - val_loss: 0.0237 - val_mean_absolute_error

In [12]:
model3=keras.models.load_model("hello.h5")
y_test_predict = model3.predict(x_test)
DataScalery.inverse_transform(y_test_predict)

array([[ 2751.37353516],
       [ 2764.99267578],
       [ 2770.78149414],
       ..., 
       [ 1392.90002441],
       [ 1395.84716797],
       [ 1393.45166016]], dtype=float32)

In [7]:
model4=keras.models.load_model("hello.h5")
model4.fit(x_train, y_train, epochs=100, batch_size=20,validation_split=0.05)
model4.save("hello.h5")
model4.summary()

Train on 36238 samples, validate on 1908 samples
Epoch 1/100
36238/36238 [==============================] - 14s - loss: 0.0099 - mean_absolute_error: 0.0758 - val_loss: 0.0208 - val_mean_absolute_error: 0.1146
Epoch 2/100
36238/36238 [==============================] - 13s - loss: 0.0100 - mean_absolute_error: 0.0762 - val_loss: 0.0248 - val_mean_absolute_error: 0.1337
Epoch 3/100
36238/36238 [==============================] - 13s - loss: 0.0100 - mean_absolute_error: 0.0760 - val_loss: 0.0203 - val_mean_absolute_error: 0.1107
Epoch 4/100
36238/36238 [==============================] - 14s - loss: 0.0099 - mean_absolute_error: 0.0757 - val_loss: 0.0210 - val_mean_absolute_error: 0.1137
Epoch 5/100
36238/36238 [==============================] - 15s - loss: 0.0098 - mean_absolute_error: 0.0756 - val_loss: 0.0192 - val_mean_absolute_error: 0.1067
Epoch 6/100
36238/36238 [==============================] - 13s - loss: 0.0099 - mean_absolute_error: 0.0760 - val_loss: 0.0220 - val_mean_absolute

36238/36238 [==============================] - 11s - loss: 0.0098 - mean_absolute_error: 0.0755 - val_loss: 0.0200 - val_mean_absolute_error: 0.1074
Epoch 52/100
36238/36238 [==============================] - 11s - loss: 0.0098 - mean_absolute_error: 0.0755 - val_loss: 0.0181 - val_mean_absolute_error: 0.1039
Epoch 53/100
36238/36238 [==============================] - 11s - loss: 0.0098 - mean_absolute_error: 0.0755 - val_loss: 0.0176 - val_mean_absolute_error: 0.0992
Epoch 54/100
36238/36238 [==============================] - 11s - loss: 0.0099 - mean_absolute_error: 0.0756 - val_loss: 0.0205 - val_mean_absolute_error: 0.1114
Epoch 55/100
36238/36238 [==============================] - 27s - loss: 0.0099 - mean_absolute_error: 0.0756 - val_loss: 0.0219 - val_mean_absolute_error: 0.1223
Epoch 56/100
36238/36238 [==============================] - 23s - loss: 0.0099 - mean_absolute_error: 0.0757 - val_loss: 0.0173 - val_mean_absolute_error: 0.0937
Epoch 57/100
36238/36238 [===============

In [8]:
y_test_predict = model4.predict(x_test)
DataScalery.inverse_transform(y_test_predict)

array([[ 2421.73974609],
       [ 2466.98120117],
       [ 2504.74633789],
       ..., 
       [ 1432.89038086],
       [ 1434.86877441],
       [ 1433.60925293]], dtype=float32)

In [6]:
DataScalery.inverse_transform(y_test)

array([[ 3000.],
       [ 7000.],
       [ 5000.],
       ..., 
       [ 1700.],
       [ 1500.],
       [ 1600.]])

In [3]:
model4=keras.models.load_model("hello.h5")
model4.

In [10]:
model4.layers

In [13]:
model4.layers[-1].output

<tf.Tensor 'dense_5/BiasAdd:0' shape=(?, 1) dtype=float32>

In [17]:
testmodel = Sequential(model4.layers)
testmodel.summary()
# testmodel.compile(loss='mse', optimizer='rmsprop',  metrics=['mae'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 40        
_________________________________________________________________
batch_normalization_1 (Batch (None, 8)                 32        
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
batch_normalization_2 (Batch (None, 8)                 32        
_________________________________________________________________
activation_2 (Activation)    (None, 8)                 0         
__________

In [ ]:
DataScalery.inverse_transform(testmodel.predict())